# Ollama Website Summarizer
## Scrape websites and summarize them locally using Ollama

This script is a complete example of the day 1 program, which uses OpenAI API to summarize websites, altered to use techniques from the day 2 exercise to call Ollama models locally.

In [ ]:
# imports

import ollama
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

MODEL = "llama3.2"

# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt
    
# Create a messages list for a summarize prompt given a website

def create_summarize_prompt(website):
    return [
        {"role": "system", "content": "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown." },
        {"role": "user", "content": user_prompt_for(website)}
    ]

# And now: call Ollama to summarize

def summarize(url):
    website = Website(url)
    messages = create_summarize_prompt(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']
    
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

## Pre-requisites

Before we can run the script above, we need to make sure Ollama is running on your machine!

Simply visit ollama.com and install!

Once complete, the ollama server should already be running locally.
If you visit:
http://localhost:11434/

You should see the message Ollama is running.

## Run!

Shift+Enter the code below to summarize a website.

### NOTE!

This will only work with websites that return HTML content, and may return unexpected results for SPAs that are created with JS.

In [ ]:
display_summary("https://edwarddonner.com/2024/11/13/llm-engineering-resources/")

Edit the URL above, or add code blocks of your own to try it out!